# Face Recognition using KNN
### Data Collection:
#### Phase-I:
1. Write a Python Script that captures images from your webcam video stream
2. Extract all Faces from the image frame (Using haarcascades)
3. Store the face information into numpy arrays

#### Phase-II:
1. Read and show video stream, capture images
2. Detect faces and show the bounding box
3. Flatten the largest face image and save in a numpy array
4. Repeat the above for multiple person to generate training data


In [6]:
# import libraries
import cv2
import numpy as np

# Initialize Camera
cap = cv2.VideoCapture(0)

# Face Detection
face_cascade = cv2.CascadeClassifier(
    'haarcascades/haarcascade_frontalface_alt.xml')
skip = 0
face_data = []
dataset_path = './data/'
filename = input("Enter the name of the person: ")
while True:
    # capture the frame (photo)
    ret, frame = cap.read()

    if ret == False:
        continue

    # Convert captured frame to gray scale (single colour channel)
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    faces = sorted(faces, key=lambda f: f[2]*f[3], reverse=True)


    # pick the last face (because it is the largest face according to area(f[2]*f[3]))
    for face in faces[-1:]:
        x, y, w, h = face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)

        # Extract ROI (Region of Interest)
        offset = 10
        face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset]
        face_section = cv2.resize(face_section, (100, 100))

        skip += 1
        if skip % 10 == 0:
            face_data.append(face_section)
            print(len(face_data))

    cv2.imshow("Frame", frame)
    cv2.imshow("Face Section", face_section)

    # Store every 10th face
    key_pressed = cv2.waitKey(1) & 0xFF
    # if user presses 'q' then break exit from the loop
    if key_pressed == ord('q'):
        break
# Convert our face list array into a numpy array
face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0], -1))
print(face_data.shape)

# save this data into file system
np.save(dataset_path+filename+'.npy', face_data)
cap.release()
cv2.destroyAllWindows()

Enter the name of the person: test
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
(40, 30000)


# Recognise faces using KNN

### Steps:
1. Read a video stream using opencv.
2. Extract faces out of it.
3. Load the training data (numpy arrays of all the persons).
  - X - values are stored in the numpy arrays
  - y - values we need to assign for each person
4. Use KNN to find the prediction of face (ini)
5. Map the predicted id to the name of the user
6. Display the prediction on the screen - bounding box and name.

In [1]:
# KNN implementation from scratch
def distance(x1, x2):
    return np.sqrt(sum((x1 - x2)**2))


def knn(train, test, k=5):
    # Empty list
    dist = []

    for i in range(train.shape[0]):
        # Get the vector and label
        ix = train[i, :-1]
        iy = train[i, -1]

        # Compute the distance from test poing
        d = distance(test, ix)
        dist.append([d, iy])

    # sort based on distance from test point
    dk = sorted(dist, key=lambda x: x[0])[:k]
    # Retrieve only the labels
    labels = np.array(dk)[:, -1]

    # Get frequencies of each label
    output = np.unique(labels, return_counts=True)

    # find the max frequency and corresponding label
    index = np.argmax(output[1])

    return output[0][index]

In [9]:
import os
import cv2
import numpy as np

# init camera
cap = cv2.VideoCapture(0)

# Face Detection
face_cascade = cv2.CascadeClassifier(
    "haarcascades/haarcascade_frontalface_alt.xml")

skip = 0
dataset_path = './data/'

face_data = []
labels = []

# labels for given file
class_id = 0
# mapping between id and name
names = {}

# Data Preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        # create a mapping between class_id and name
        names[class_id] = fx[:-4]
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)

        # create labels for class
        target = class_id * np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data, axis=0)
face_labels = np.concatenate(labels, axis=0).reshape((-1, 1))

train_set = np.concatenate((face_dataset, face_labels), axis=1)

# Testing

while True:
    ret, frame = cap.read()
    
    if ret is None:
        continue

    faces = face_cascade.detectMultiScale(frame, 1.3, 5)

    for face in faces:
        x, y, w, h = face

        # Get the face ROI
        offset = 10
        face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset]
        face_section = cv2.resize(face_section, (100, 100))

        # predicted label (out)
        out = knn(train_set, face_section.flatten())

        # Display the name and the rectangle around the face
        pred_name = names[int(out)]
        cv2.putText(frame, pred_name, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('dark_background')

a = np.arange(0, 20)
b = a**2
plt.plot(a, b)
plt.title("This is not showing")